In [1]:
from pyspark.sql import SparkSession
def init_cluster()-> SparkSession:
    spark = SparkSession.builder \
        .appName("Carga Coronavirus") \
        .master("local[4]") \
        .config("spark.executor.memory", "8g") \
        .config("spark.sql.execution.arrow.enabled", "true") \
        .getOrCreate()
    return spark

In [2]:
import glob
archivos = glob.glob('..\data\*.csv')
archivos

['..\\data\\country_wise_latest.csv',
 '..\\data\\covid_19_clean_complete.csv',
 '..\\data\\day_wise.csv',
 '..\\data\\full_grouped.csv',
 '..\\data\\usa_county_wise.csv',
 '..\\data\\worldometer_data.csv']

In [31]:
from datetime import date
data = {}
for archivo in archivos:
    df = spark.read.options(header=True, inferSchema=True).csv(archivo)
    data[archivo.split('\\')[-1]] = df
data

{'country_wise_latest.csv': DataFrame[Country/Region: string, Confirmed: int, Deaths: int, Recovered: int, Active: int, New cases: int, New deaths: int, New recovered: int, Deaths / 100 Cases: double, Recovered / 100 Cases: double, Deaths / 100 Recovered: string, Confirmed last week: int, 1 week change: int, 1 week % increase: double, WHO Region: string],
 'covid_19_clean_complete.csv': DataFrame[Province/State: string, Country/Region: string, Lat: double, Long: double, Date: timestamp, Confirmed: int, Deaths: int, Recovered: int, Active: int, WHO Region: string],
 'day_wise.csv': DataFrame[Date: timestamp, Confirmed: int, Deaths: int, Recovered: int, Active: int, New cases: int, New deaths: int, New recovered: int, Deaths / 100 Cases: double, Recovered / 100 Cases: double, Deaths / 100 Recovered: double, No. of countries: int],
 'full_grouped.csv': DataFrame[Date: timestamp, Country/Region: string, Confirmed: int, Deaths: int, Recovered: int, Active: int, New cases: int, New deaths: i

In [33]:
from pyspark.sql.functions import col
data['country_wise_latest.csv'].replace({'inf':'0.0'},'Deaths / 100 Recovered')
data['country_wise_latest.csv'].withColumn("Deaths / 100 Recovered",col("Deaths / 100 Recovered").cast("double"))
data['country_wise_latest.csv'].sort('Deaths / 100 Recovered',ascending=True).show(4)

+--------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+---------------+
|Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|     WHO Region|
+--------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+---------------+
|      Dominica|       18|     0|       18|     0|        0|         0|            0|               0.0|                100.0|                   0.0|                 18|            0|              0.0|       Americas|
|     Greenland|       14|     0|       13|     1|        1|         0|            0|               0.0|                92.86|  

In [72]:
for i in data:
    print(data[i].dtypes)
    print(data[i].show(5))


[('Country/Region', 'string'), ('Confirmed', 'int'), ('Deaths', 'int'), ('Recovered', 'int'), ('Active', 'int'), ('New cases', 'int'), ('New deaths', 'int'), ('New recovered', 'int'), ('Deaths / 100 Cases', 'double'), ('Recovered / 100 Cases', 'double'), ('Deaths / 100 Recovered', 'string'), ('Confirmed last week', 'int'), ('1 week change', 'int'), ('1 week % increase', 'double'), ('WHO Region', 'string')]
+--------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|
+--------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+------------------